# 修饰符

## 函数是一种对象

在 `Python` 中，函数是也是一种对象。

In [1]:
def foo(x):
    print( x)
    
print(type(foo))

<class 'function'>


查看函数拥有的方法：

In [2]:
dir(foo)

['__annotations__',
 '__call__',
 '__class__',
 '__closure__',
 '__code__',
 '__defaults__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__get__',
 '__getattribute__',
 '__globals__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__kwdefaults__',
 '__le__',
 '__lt__',
 '__module__',
 '__name__',
 '__ne__',
 '__new__',
 '__qualname__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__']

在这些方法中，`__call__` 是最重要的一种方法： 

In [3]:
foo.__call__(42)

42


相当于：

In [4]:
foo(42)

42


因为函数是对象，所以函数可以作为参数传入另一个函数：

In [5]:
def bar(f, x):
    x += 1
    f(x)

In [6]:
bar(foo, 4)

5


## 修饰符

修饰符是这样的一种函数，它接受一个函数作为输入，通常输出也是一个函数：

In [5]:
def dec(f):
    print ('I am decorating function', id(f))
    return f

将 `len` 函数作为参数传入这个修饰符函数：

In [6]:
declen = dec(len)

I am decorating function 2068164243032


使用这个新生成的函数：

In [8]:
declen([10,20,30])

3

上面的例子中，我们仅仅返回了函数的本身，也可以利用这个函数生成一个新的函数，看一个新的例子：

In [9]:
def loud(f):
    def new_func(*args, **kw):
        print( 'calling with', args, kw)
        rtn = f(*args, **kw)
        print ('return value is', rtn)
        return rtn
    return new_func

In [10]:
loudlen = loud(len)

In [11]:
loudlen([10, 20, 30])

calling with ([10, 20, 30],) {}
return value is 3


3

## 用 @ 来使用修饰符

`Python` 使用 `@` 符号来将某个函数替换为修饰符之后的函数： 

例如这个函数：

In [12]:
def foo(x):
    print (x)
    
foo = dec(foo)

I am decorating function 2068238080616


可以替换为：

In [16]:
@dec
def foo(x):
    print( x)

I am decorating function 2068238081192


事实上，如果修饰符返回的是一个函数，那么可以链式的使用修饰符：

```python
@dec1
@dec2
def foo(x):
    print x
```

使用修饰符 `loud` 来定义这个函数：

In [14]:
@loud
def foo(x):
    print( x)

In [15]:
foo(42)

calling with (42,) {}
42
return value is None


## 例子

定义两个修饰器函数，一个将原来的函数值加一，另一个乘二：

In [17]:
def plus_one(f):
    def new_func(x):
        return f(x) + 1
    return new_func

def times_two(f):
    def new_func(x):
        return f(x) * 2
    return new_func

定义函数，先乘二再加一：

In [18]:
@plus_one
@times_two
def foo(x):
    return int(x)

In [19]:
foo(13)

27

## 修饰器工厂

`decorators factories` 是返回修饰器的函数，例如：

In [20]:
def super_dec(x, y, z):
    def dec(f):
        def new_func(*args, **kw):
            print( x + y + z)
            return f(*args, **kw)
        return new_func
    return dec

它的作用在于产生一个可以接受参数的修饰器，例如我们想将 `loud` 输出的内容写入一个文件去，可以这样做：

In [21]:
def super_loud(filename):
    fp = open(filename, 'w')
    def loud(f):
        def new_func(*args, **kw):
            fp.write('calling with' + str(args) + str(kw))
            # 确保内容被写入
            fp.flush()
            fp.close()
            rtn = f(*args, **kw)
            return rtn
        return new_func
    return loud

可以这样使用这个修饰器工厂：

In [22]:
@super_loud('test.txt')
def foo(x):
    print x

调用 `foo` 就会在文件中写入内容：

In [23]:
foo(12)

12


查看文件内容：

In [24]:
with open('test.txt') as fp:
    print fp.read()

calling with(12,){}


In [25]:
import os
os.remove('test.txt')

## 修饰符使用

### @classmethod 修饰符

在 `Python` 标准库中，有很多自带的修饰符，例如 `classmethod` 将一个对象方法转换为类方法： 

In [21]:
class Foo(object):
    @classmethod
    def bar(cls, x):
        print ('the input is', x)
        
    def __init__(self):
        pass


类方法可以通过 `类名.方法` 来调用：

In [22]:
Foo.bar(12)

the input is 12


### @property 修饰符

有时候，我们希望像 __Java__ 一样支持 `getters` 和 `setters` 的方法，这时候就可以使用 `property` 修饰符：

In [23]:
class Foo(object):
    def __init__(self, data):
        self.data = data
    
    @property
    def x(self):
        return self.data

此时可以使用 `.x` 这个属性查看数据（不需要加上括号）：

In [26]:
foo = Foo(23)
foo.x

23

这样做的好处在于，这个属性是只读的

In [27]:
foo.x = 1

AttributeError: can't set attribute

如果想让它变成可读写，可以加上一个修饰符 `@x.setter`：

In [28]:
class Foo(object):
    def __init__(self, data):
        self.data = data
    
    @property
    def x(self):
        return self.data
    
    @x.setter
    def x(self, value):
        self.data = value

In [29]:
foo = Foo(23)
foo.x

23

In [30]:
foo.x = 20
foo.x

20

### Numpy 的 @vectorize 修饰符

In [32]:
import numpy as np
from numpy import vectorize, arange

将函数向量化

In [33]:
@vectorize
def f(x):
    if x <= 0:
        return x
    else:
        return 0

f(arange(-10.0,10.0))

array([-10.,  -9.,  -8.,  -7.,  -6.,  -5.,  -4.,  -3.,  -2.,  -1.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.])

### 注册一个函数

来看这样的一个例子，定义一个类

In [34]:
class Registry(object):
    def __init__(self):
        self._data = {}
    def register(self, f, name=None):
        if name == None:
            name = f.__name__
        self._data[name] = f
        setattr(self, name, f)

`register` 方法接受一个函数，将这个函数名作为属性注册到对象中。

产生该类的一个对象：

In [35]:
registry = Registry()

使用该对象的 `register` 方法作为修饰符：

In [36]:
@registry.register
def greeting():
    print ("hello world")

这样这个函数就被注册到 `registry` 这个对象中去了：

In [37]:
registry._data

{'greeting': <function __main__.greeting()>}

In [38]:
registry.greeting

<function __main__.greeting()>

### @wraps

一个通常的问题在于:

In [39]:
def logging_call(f):
    def wrapper(*a, **kw):
        print( 'calling {}'.format(f.__name__))
        return (f(*a, **kw))
    return wrapper

@logging_call
def square(x):
    '''
    square function.
    '''
    return x ** 2

print (square.__doc__, square.__name__)

None wrapper


我们使用修饰符之后，`square` 的 `metadata` 完全丢失了，返回的函数名与函数的 `docstring` 都不对。

一个解决的方法是从 `functools` 模块导入 `wraps` 修饰符来修饰我们的修饰符：

In [40]:
import functools

def logging_call(f):
    @functools.wraps(f)
    def wrapper(*a, **kw):
        print ('calling {}'.format(f.__name__))
        return f(*a, **kw)
    return wrapper

@logging_call
def square(x):
    '''
    square function.
    '''b
    return x ** 2

print (square.__doc__, square.__name__)


    square function.
     square


### Class 修饰符

与函数修饰符类似，类修饰符是这样一类函数，接受一个类作为参数，通常返回一个新的类